In [1]:
import numpy as np
from calibration import CalibrationSystem, SparseModel
import gpflow
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
from calibration.errormetrics import MAE, MSE, NMSE, NLPD, compute_test_data
from calibration.synthetic import generate_synthetic_dataset, getstaticsensortranform, getmobilesensortranform
from GPyOpt.methods import BayesianOptimization
import pickle
from networkx import NodeNotFound
from calibration.simple import compute_simple_calibration, compute_simple_predictions


In [ ]:
def f(x):
    staticls = np.exp((np.log(24*1000)-np.log(24*5))*x[0][0]+np.log(24*5))
    mobilels = np.exp((np.log(24*1000)-np.log(24*5))*x[0][1]+np.log(24*5))
    likelihoodstd = np.exp((np.log(150)-np.log(1))*x[0][2]+np.log(1))
    kernelscale = np.exp((np.log(150)-np.log(1))*x[0][3]+np.log(1))
    
    kstatic = gpflow.kernels.RBF(kernelscale,staticls) + gpflow.kernels.Bias(2)
    kmobile = gpflow.kernels.RBF(kernelscale,mobilels) + gpflow.kernels.Bias(2)
    jitter = 1e-4
    while True:
        print("...")
        try:
            ###LR = 0.01 -> 0.04?!
            cs = CalibrationSystem(augmentedX, augmentedY, Z, refsensor, 1, transform_fn, transform_fn_loggrad, [kstatic,kmobile], kernelindices,lr=0.01,likelihoodstd=likelihoodstd,minibatchsize=50,jitter=jitter)
            
        
            import time
            before = time.time()
            print("Starting Run")
            elbo_record = cs.run(1000,samples=100) #1000,100
            print(time.time()-before)

            testX, testY, testtrueY = compute_test_data(X,Y,trueY,refsensor)

            testsm = SparseModel(testX,cs.Z,C,cs.k)
            qf_mu,qf_cov = testsm.get_qf(cs.mu,cs.scale)
            #qf_mu=qf_mu*0
            predY = transform_fn(qf_mu[None,:,:],testY[:,0:1],None).numpy()[:,:,0].T
            #predY[predY>100]=100
            nlpd = NLPD(np.log(testtrueY),np.log(testY[:,0:1])+qf_mu,np.sqrt(np.diag(qf_cov))[:,None])
            nmse = NMSE(testtrueY,predY)
            mse = MSE(testtrueY,predY)
            mae = MAE(testtrueY,predY)
            break
        except tf.errors.InvalidArgumentError:
            print("EXCEPTION: INCREASING JITTER!")
            jitter = jitter * 10
            if jitter>1: 
                print("Jitter > 1, giving up, returning NMSE = 1")
                return 1 #we'll just give up here.
            print("New Jitter:",jitter)
    print("staticls=%5.2f, mobilels=%5.2f likelihoodstd=%5.2f: nlpd=%5.2f nmse=%5.2f mse=%5.2f mae=%5.2f" % (staticls,mobilels,likelihoodstd,nlpd,nmse,mse,mae))
    global searchresults
    searchresults.append({'staticls':staticls,'mobilels':mobilels,'likelihoodstd':likelihoodstd,'kernelscale':kernelscale,'nlpd':nlpd,'nmse':nmse,'mse':mse,'mae':mae})
    print(searchresults)
    return nmse

def plotsearchresults(searchresults):
    mls = []
    lstd = []
    nmse = []
    sls = []
    for sr in searchresults:
        mls.append(sr['mobilels'])
        lstd.append(sr['likelihoodstd'])
        nmse.append(sr['nmse'])
        sls.append(sr['staticls'])
    nmse=np.array(nmse)
    sls = np.array(sls)    
    plt.scatter(sls,mls,5+(nmse-np.min(nmse))*15000,facecolor='none',edgecolor='black')
    #plt.plot(sls,mls,'-k',alpha=0.1)
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('static lengthscale / hours')
    plt.ylabel('mobile lengthscale / hours')
    #plt.scatter(1726.1398659,2.65914794,1000,marker='x')

for randomrestart in range(10):
    for noisescale in [20,10,5,2]:
        print("============================================")
        print("Noise Scale: %d" % noisescale)
        Nstatic = 10
        Nmobile = 4
        Ttotal = 24*180
        Nrefs = 4
        Nvisitsperdayref = 0.5
        Nvisitsperday = 2.5
        staticsensornoise = noisescale
        mobilesensornoise = noisescale
        Nsamps = 1
        refsensor,X,Y,trueY,statics,mobilecentres = generate_synthetic_dataset(Nstatic, Nmobile, Ttotal, Nrefs, Nvisitsperdayref, Nvisitsperday, staticsensornoise, mobilesensornoise, Nsamps)

        #Plot locations of sensors
        plt.figure(figsize=[8,8])
        plt.plot(statics[:Nrefs,0],statics[:Nrefs,1],'+g',label='Reference sensors',markersize=10,mew=3)
        plt.plot(statics[Nrefs:,0],statics[Nrefs:,1],'x',label='Other static sensors',mew=3)
        plt.plot(mobilecentres[:,0],mobilecentres[:,1],'ro',label='mobile sensors')
        plt.legend()
        plt.axis('equal')
        plt.savefig('synthetic_sensorplacement_%d_%d.pdf' % (noisescale,randomrestart))

        ##Plot graphs of synthetic data.
        plt.figure(figsize=[8,8])
        pltidx = 0
        for n in range(Nstatic+Nmobile):
            keep = X[:,1]==n
            if n<Nrefs: continue
            pltidx+=1
            plt.subplot(5,2,pltidx)
            plt.plot(X[keep,0],Y[keep][:,0]/trueY[keep],'.')

            keep = X[:,2]==n
            plt.plot(X[keep,0],Y[keep][:,1]/trueY[keep],'.')
            plt.xlim([0,4500])
            plt.ylim([0,2.4])
            if pltidx<=8:
                #plt.gca().axes.get_xaxis().set_visible(False)
                plt.xticks(np.arange(0,5000,1000),['']*5)
            else:
                plt.xlabel('Hour')
            if pltidx%2==0:
                #plt.gca().axes.get_yaxis().set_visible(False)
                pass
            else:
                plt.ylabel('Scaling')
            r = []
            Ts = np.arange(0,Ttotal,1)
            for t in Ts:
                if n<Nstatic:
                    r.append(getstaticsensortranform(t,100,4,n,0))
                else:
                    r.append(getmobilesensortranform(t,100,n-Nstatic,0))
            plt.grid()
            plt.plot(Ts,np.array(r)/100,'k-')
        plt.savefig('scalingsyntheticdata_%d_%d.pdf' % (noisescale,randomrestart))

        ##Compute sensor stats
        msperiod = []
        ssperiod = []
        for sensor in range(0,4):
            p = (2*np.pi)/(0.01*(1+0.5*np.cos(4+sensor)))
            #print(sensor,p)
            msperiod.append(p)
        print("---")
        for sensor in range(4,10):
            p = (2*np.pi)/(0.003*(1+np.cos(sensor)))
            print(sensor,p)
            ssperiod.append(p)

        print("Static sensor mean: %0.1f" % np.exp(np.mean(np.log(ssperiod))))
        print("Mobile sensor mean: %0.1f" % np.exp(np.mean(np.log(msperiod))))

        print("Static sensor median: %0.1f" % np.median(ssperiod))
        print("Mobile sensor median: %0.1f" % np.median(msperiod))


        ##Resampling data
        #A hacky experiment to encourage reference data to be used more...
        #otherwise most minibatches won't include any reference signal?
        #is this really dubious...?
        print("%0.1f%% of colocations are with a reference sensor" % (100*np.mean(np.any(np.isin(X[:,1:],np.where(refsensor)[0]),1))))
        refencounters = np.any(np.isin(X[:,1:],np.where(refsensor)[0]),1)
        augmentedX = np.r_[X,np.repeat(X[refencounters,:],10,0)]
        augmentedY = np.r_[Y,np.repeat(Y[refencounters,:],10,0)]
        print("After augmentation...")
        print("%0.1f%% of colocations are with a reference sensor" % (100*np.mean(np.any(np.isin(augmentedX[:,1:],np.where(refsensor)[0]),1))))

        #Place inducing points
        Z = np.linspace(-200,Ttotal+200,60)[:,None] ##60

        C = 1
        def transform_fn(samps,Y,sideY):
            return Y*tf.exp(samps[:,:,0:1])

        def transform_fn_loggrad(samps,Y,sideY):
            return samps[:,:,0:1]

        kernelindices = [[0]*Nstatic+[1]*Nmobile]

        ## Hyperparameter Search using Bayesian Optimisation
        #or load...
        if False:
            searchresults = []
            domain = [{'name': 'var_1', 'type': 'continuous', 'domain': (0,1), 'dimensionality':4}]
            myBopt = BayesianOptimization(f=f, domain=domain)
            myBopt.run_optimization(max_iter=30)
            pickle.dump(searchresults,open('searchresults_%d.p' % noisescale,'wb'))
            #searchresults = pickle.load(open('searchresults_%d.p' % noisescale,'rb'))

            #Plot the bayesian optimisation
            plt.figure(figsize=[5,5])
            plotsearchresults(searchresults)
            plt.savefig('BayesianOptimisationSyntheticData_%d_%d.pdf' % (noisescale,randomrestart))

            minnmse = np.inf
            bestresult = None
            for sr in searchresults:
                if sr['nmse']<minnmse:
                    minnmse = sr['nmse']
                    bestresult = sr
            for s in bestresult: print("%20s: %0.5f" % (s,bestresult[s]))

        # Then let's solve using a good solution
        #this seems to work ok (chosen using noise-scale of 10)
        kstatic = gpflow.kernels.RBF(7.645,2201) + gpflow.kernels.Bias(2)#7.645)
        kmobile = gpflow.kernels.RBF(7.645,581) + gpflow.kernels.Bias(2)#7.645)
        cs = CalibrationSystem(augmentedX, augmentedY, Z, refsensor, 1, transform_fn, transform_fn_loggrad, [kstatic,kmobile], kernelindices,lr=0.01,likelihoodstd=6.02,minibatchsize=100)
        import time
        before = time.time()
        print("Starting Run")
        elbo_record = cs.run(3000,samples=200,verbose=True) ##3000 iterations, 200 samples
        print(time.time()-before)

        plt.figure()
        plt.plot(elbo_record)
        plt.yscale('log')

        ###################################
        ###########Simple approach#########
        testX, testY, testtrueY = compute_test_data(X,Y,trueY,refsensor)

        ## First optimise weightovertime and delta
        if True: #False:
            bestcomb = None
            bestnmse = np.inf
            for weightovertime in np.logspace(-2,2,10,base=np.e):
                for delta in np.logspace(np.log(24*3),np.log(24*200),10,base=np.e):
                    #print(weightovertime,delta)
                    #try:

                    G,allsp,allcals,allcallists,allpopts,allpcovs,allpoptslists = compute_simple_calibration(X,Y,delta,refsensor,weightovertime=weightovertime,mincolocationsinperiod=1)
                    preds,res2,res = compute_simple_predictions(testX,testY,testtrueY,allcals,delta)
                    nmse = NMSE(testtrueY[:,0],preds[:,0])
                    if nmse<bestnmse:
                        bestnmse=nmse
                        bestcomb = (weightovertime,delta)
                    print("%5.2f, %5.2f %5.3f" % (weightovertime,delta,nmse))
        print("Best:")
        print("bestcomb:",bestcomb)
        print("bestnmse:",bestnmse)
        weightovertime=bestcomb[0]#0.135 #0.367
        delta = bestcomb[1] #4800#hours
        G,allsp,allcals,allcallists,allpopts,allpcovs,allpoptslists = compute_simple_calibration(X,Y,delta,refsensor,weightovertime=weightovertime,mincolocationsinperiod=1)
        preds,res2,res = compute_simple_predictions(testX,testY,testtrueY,allcals,delta)


        print("Using no method!")
        nmse = NMSE(testtrueY[:,0],testY[:,0])
        mse = MSE(testtrueY[:,0],testY[:,0])
        mae = MAE(testtrueY[:,0],testY[:,0])
        print("nmse=%5.5f mse=%5.2f mae=%5.2f" % (nmse,mse,mae))

        print("Using the graph method:")
        nmse = NMSE(testtrueY[:,0],preds[:,0])
        mse = MSE(testtrueY[:,0],preds[:,0])
        mae = MAE(testtrueY[:,0],preds[:,0])
        print("nmse=%5.5f mse=%5.2f mae=%5.2f" % (nmse,mse,mae))


        testsm = SparseModel(testX,cs.Z,1,cs.k)
        qf_mu,qf_cov = testsm.get_qf(cs.mu,cs.scale)
        #qf_mu=qf_mu*0
        predY = transform_fn(qf_mu[None,:,:],testY[:,0:1],None).numpy()[:,:,0].T

        nlpd = NLPD(np.log(testtrueY[:,0]),np.log(testY[:,0:1])+qf_mu[:,0],np.sqrt(np.diag(qf_cov)))
        keep = ~np.isnan(testtrueY)
        nmse = NMSE(testtrueY[:,0],predY[:,0])
        mse = MSE(testtrueY[:,0],predY[:,0])
        mae = MAE(testtrueY[:,0],predY[:,0])
        print("Using the variational method")
        print("nlpd=%5.2f nmse=%5.5f mse=%5.2f mae=%5.2f" % (nlpd,nmse,mse,mae))

        ## Plot encounters
        plt.figure(figsize=[20,4])
        plt.vlines(X[:,0],X[:,1],X[:,2])
        plt.xlim([500.0,850.0])
        plt.hlines(np.array([0,1,2,3]),-1000,5000,'b')
        plt.hlines(np.array([13,12,11,10]),-1000,5000,'r')
        plt.hlines(np.arange(4,13),-1000,5000,'g',alpha=0.4)
        plt.vlines(np.arange(0,3000,24),-4,20,'grey',alpha=0.5)
        plt.ylim([-0.5,13.5])
        plt.yticks(np.arange(14),np.arange(14));
        plt.xticks(np.arange(int(500/24)*24,850,24),np.arange(int(500/24)*24,850,24).astype(int))
        plt.ylabel('Sensor')
        plt.xlabel('Time / hours')
        plt.savefig('syntheticvisits_%d_%d.pdf' % (noisescale,randomrestart))

        ##Plot variational approach results
        C = 1
        pltn=1
        plt.figure(figsize=[10,10])
        for si,refs in enumerate(refsensor):
            if refs: continue
            x = np.linspace(0,Ttotal,160)
            testX = np.zeros([0,3])
            for ci in range(C):
                tempX = np.c_[x,np.ones_like(x)*si,np.full_like(x,ci)]
                testX = np.r_[testX,tempX]#.astype(int)
            testsm = SparseModel(testX,cs.Z,C,cs.k)
            qf_mu,qf_cov = testsm.get_qf(cs.mu,cs.scale)
            if cs.mulike is not None:
                qf_mulike,qf_covlike = testsm.get_qf(cs.mulike,cs.scalelike)
                sampslike = testsm.get_samples_one_sensor(cs.mulike,cs.scalelike)
            samps = testsm.get_samples_one_sensor(cs.mu,cs.scale)

            #plt.figure(figsize=[14,7])
            plt.subplot(5,2,pltn)
            pltn+=1
            plt.plot(x,1/tf.exp(qf_mu[:,0]),'k-',lw=2)
            #plt.plot(x,1/np.exp(samps[:,:,0].numpy().T),'k.',alpha=0.005);

            plt.plot(x,1/tf.exp((qf_mu[:,0]+2*np.sqrt(np.diag(qf_cov)[:]))),'k-',alpha=0.5)
            plt.plot(x,1/tf.exp((qf_mu[:,0]-2*np.sqrt(np.diag(qf_cov)[:]))),'k-',alpha=0.5)

            senseX = (X[:,1]==si) & (X[:,2]<Nrefs)
            plt.plot(X[senseX,0],Y[senseX,0]/Y[senseX,1],'xr',markersize=12,mew=2)



            senseX = (X[:,2]==si) & (X[:,1]<Nrefs)
            plt.plot(X[senseX,0],Y[senseX,1]/Y[senseX,0],'xr',markersize=12,mew=2)
            senseX = (X[:,1]==si)
            plt.plot(X[senseX,0],Y[senseX,0]/Y[senseX,1],'.g',markersize=4)
            senseX = (X[:,2]==si)
            plt.plot(X[senseX,0],Y[senseX,1]/Y[senseX,0],'.g',markersize=4)    
            #plotbars(np.arange(0,np.max(X[:,0]),delta),delta,np.exp(allscales[:,si]),'y')
            plt.ylim([0,3])
            #plt.yscale('log')
            plt.grid()
            if si<Nstatic:
                for samp in range(10):
                    plt.plot([(getstaticsensortranform(t,100.0,Nrefs,si,noisescale=0)/100) for t in np.arange(Ttotal)],'b-',alpha=0.1)
            else:
                for samp in range(10):
                    plt.plot([(getmobilesensortranform(t,100.0,si-Nstatic,noisescale=0)/100) for t in np.arange(Ttotal)],'b-',alpha=0.1)
        plt.savefig('results_synth_%d_%d.pdf' % (noisescale,randomrestart))

        ##Plot simple graph approach results
        plt.figure(figsize=[10,10])
        pltn=1
        for si,refs in enumerate(refsensor):
            if refs: continue
            plt.subplot(5,2,pltn)
            pltn+=1
            for i,t in enumerate(np.arange(0,np.max(X[:,0]),delta)):
                try:
                    plt.hlines(1/(np.exp(allcals[(si,i)])),t,t+delta)
                except KeyError:            
                    pass
            senseX = (X[:,1]==si) & (X[:,2]<Nrefs)
            plt.plot(X[senseX,0],Y[senseX,0]/Y[senseX,1],'xr',markersize=12,mew=1)



            senseX = (X[:,2]==si) & (X[:,1]<Nrefs)
            plt.plot(X[senseX,0],Y[senseX,1]/Y[senseX,0],'xr',markersize=12,mew=1)
            senseX = (X[:,1]==si)
            plt.plot(X[senseX,0],Y[senseX,0]/Y[senseX,1],'.g',markersize=3)
            senseX = (X[:,2]==si)
            plt.plot(X[senseX,0],Y[senseX,1]/Y[senseX,0],'.g',markersize=3)    
            #plotbars(np.arange(0,np.max(X[:,0]),delta),delta,np.exp(allscales[:,si]),'y')
            plt.ylim([0,2.5])
            plt.xlim([0,4300])
            #plt.yscale('log')
            plt.grid()
            if si<Nstatic:
                for samp in range(10):
                    plt.plot([(getstaticsensortranform(t,100.0,Nrefs,si,noisescale=0)/100) for t in np.arange(Ttotal)],'b-',alpha=0.1)
            else:
                for samp in range(10):
                    plt.plot([(getmobilesensortranform(t,100.0,si-Nstatic,noisescale=0)/100) for t in np.arange(Ttotal)],'b-',alpha=0.1)        
            if pltn>=9:
                plt.xlabel('Time / hours')
            if pltn%2==0:
                plt.ylabel('Scaling')
            plt.savefig('results_synth_simple_%d_%d.pdf' % (noisescale,randomrestart))

At the moment I just saved the cell output (sorry...) ran this regexp on the file:

`grep "Noise\|nmse\|Using" "DONE Final Paper Synthetic Output.txt" > "DONE Final Paper Synthetic Summary.txt"`

In [103]:
import re
patterns = []
patterns.append(re.compile("nlpd=([ 0-9.]*)"))
patterns.append(re.compile("nmse=([ 0-9.]*)"))
patterns.append(re.compile(" mse=([ 0-9.]*)"))
patterns.append(re.compile(" mae=([ 0-9.]*)"))

currentmethod = -1
currentnoise = -1
noisescales = {2:0,5:1,10:2,20:3}
invnoisescales = {0:2,1:5,2:10,3:20}
#results = np.full([10,4],np.nan)
results = [[[[] for _ in range(4)] for _ in range(3)] for _ in range(4)]

methodstr = ['no method','the graph','the varia']

for line in open('DONE Final Paper Synthetic Summary.txt'):
    for i,ms in enumerate(methodstr):
        if line[:15]=='Using '+ms:
            #print(line)
            currentmethod = i
    res = re.findall('Noise Scale: ([0-9]*)',line)
    if len(res)>0:
        currentnoise = noisescales[int(res[0])]
    for i,pattern in enumerate(patterns):
        #print(pattern,line)
        res = re.findall(pattern, line)
        if len(res)==0: continue
        #print(res[0])
        #print(currentmethod,i)
        results[currentnoise][currentmethod][i].append(float(res[0]))
        

In [113]:
for metric,metricname,units,prec in zip([1,2,3],['NMSE','MSE','MAE'],['',' / $(\mu g\; m{}^{-3})^2$',' / $\mu g\; m{}^{-3}$'],[2,0,1]):
    print(" ")
    print("-------%d------" % metric)
    row = ["Noise Scale / $\mu g\; m{}^{-3}$","\multicolumn{3}{c}{%s%s}" % (metricname,units),"NLPD"]
    print(" & ".join(row)+" \\\\")
    row = ["","No Method","Multi-hop","Cal. Pair Model","Cal. Pair Model"]
    print(" & ".join(row)+" \\\\")
    for noisescale in range(4):
        row = []
        row.append("%d" % invnoisescales[noisescale])
        for method in range(3):
            r = results[noisescale][method][metric]
            if prec==0: row.append("%0.0f (%0.0f)" % (np.mean(r),np.std(r)/np.sqrt(len(r))))
            if prec==1: row.append("%0.1f (%0.1f)" % (np.mean(r),np.std(r)/np.sqrt(len(r))))
            if prec==2: row.append("%0.2f (%0.2f)" % (np.mean(r),np.std(r)/np.sqrt(len(r))))
        r = results[noisescale][2][0]
        row.append("%0.1f (%0.1f)" % (np.mean(r),np.std(r)/np.sqrt(len(r))))

        print(" & ".join(row)+" \\\\")

 
-------1------
Noise Scale / $\mu g\; m{}^{-3}$ & \multicolumn{3}{c}{NMSE} & NLPD \\
 & No Method & Multi-hop & Cal. Pair Model & Cal. Pair Model \\
2 & 0.77 (0.02) & 0.27 (0.01) & 0.18 (0.01) & 42.6 (2.8) \\
5 & 0.78 (0.02) & 0.44 (0.03) & 0.21 (0.01) & 49.5 (2.3) \\
10 & 0.97 (0.02) & 0.87 (0.07) & 0.35 (0.01) & 58.2 (2.3) \\
20 & 1.46 (0.02) & 3.18 (0.69) & 0.84 (0.02) & 87.7 (1.5) \\
 
-------2------
Noise Scale / $\mu g\; m{}^{-3}$ & \multicolumn{3}{c}{MSE / $(\mu g\; m{}^{-3})^2$} & NLPD \\
 & No Method & Multi-hop & Cal. Pair Model & Cal. Pair Model \\
2 & 410 (16) & 146 (10) & 93 (6) & 42.6 (2.8) \\
5 & 431 (8) & 244 (18) & 117 (5) & 49.5 (2.3) \\
10 & 525 (7) & 470 (36) & 188 (4) & 58.2 (2.3) \\
20 & 788 (9) & 1714 (359) & 453 (6) & 87.7 (1.5) \\
 
-------3------
Noise Scale / $\mu g\; m{}^{-3}$ & \multicolumn{3}{c}{MAE / $\mu g\; m{}^{-3}$} & NLPD \\
 & No Method & Multi-hop & Cal. Pair Model & Cal. Pair Model \\
2 & 15.6 (0.3) & 8.4 (0.3) & 6.8 (0.2) & 42.6 (2.8) \\
5 & 16

In [69]:
float(r[0])

1.50131